# <font color=#005656><div align="center">Sales analysis

## Carga de librerías

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

## Preparar los datos

En vista de que se neceita la información de todos los archivos disponibles, primero se buscará anexarlos para facilitar el análisis.

- La función os.listdir() otorga una lista de todos los archivos disponibles dentro de una carpeta, el método map crea un iteración para un funución específica.
- El método reset_index() ayuda a crear una columna adicional que indique la procedencia de cada fila en el dataframe resultante.

**Carga de datos**

In [ ]:
# Método alternativo usando el método for

mainpath = r"C:\Users\DELL\OneDrive\Formación\Python\GITHUB - Repositorios\Case analysis\1. Análisis de ventas\Sales_Data"
# Almacenar el nombre de cada archivo según una dirección
os.listdir(mainpath)

# Declaración de un Dataframe vacío
all_months_data = pd.DataFrame()

# Carga de todos los archivos en un dirección específica
for file in files:
    df = pd.read_csv(os.path.join(mainpath,file))
    df.insert(loc=len(df.columns),
              column='File_origin',
             value=file)
    all_months_data = pd.concat([all_months_data,df])

# Creación de un nuevo archivo csv con los datos combinados
all_months_data.to_csv("all_data.csv",index = False)

all_months_data.shape

In [3]:
mainpath = r'C:\Users\DELL\OneDrive\Formación\Python\GITHUB - Repositorios\Case analysis\1. Análisis de ventas\Sales_Data'

# Crear una lista de direcciones
files = [os.path.join(mainpath,i) for i in os.listdir(mainpath)]

# Función personalizada de carga de datos
def custom_read_csv(x_path):
    return pd.read_csv(filepath_or_buffer = x_path,
                       sep = ',',
                       skip_blank_lines = True,
                       encoding = 'utf-8')

df = pd.concat(objs = map(custom_read_csv, files),
               keys = os.listdir(mainpath),
               names = ['File']).reset_index(level = ['File'])

#df.reset_index(inplace = True, drop = True)

#df.info()

En este punto ya se tiene la información reunida en un solo dataframe. Ahora deben ser tranformados para su análisis de acuerdo con una serie de pasos.
1. Eliminación de filas con valores faltantes en la columna "Order ID".
2. Eliminación de títulos de columnas innecesarios.
3. Asignación de tipos de columnas.
4. Exportar archivo resultante (opcional)

In [4]:
# T-01
df.dropna(subset = ['Order ID'], inplace = True)
# T-02
df = df.loc[df['Order ID'] != 'Order ID'].reset_index(drop = True)
# T-03
df = df.apply(pd.to_numeric, errors='ignore')

In [ ]:
# T-04
df.index.name = 'ID'
df.to_csv('Data.csv')

Las filas representa una venta según un número de orden, por lo que esta columna tinene valores repetidos. Es posible observar la cantidad de veces 

In [5]:
df.loc[df['Order ID'].duplicated(keep = False)].sort_values(by = 'Order ID')

,File,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
67262,Sales_January_2019.csv,141275,USB-C Charging Cable,1,11.95,01/07/19 16:06,"610 Walnut St, Austin, TX 73301"
67263,Sales_January_2019.csv,141275,Wired Headphones,1,11.99,01/07/19 16:06,"610 Walnut St, Austin, TX 73301"
67279,Sales_January_2019.csv,141290,AA Batteries (4-pack),3,3.84,01/02/19 08:25,"4 1st St, Los Angeles, CA 90001"
67278,Sales_January_2019.csv,141290,Apple Airpods Headphones,1,150.00,01/02/19 08:25,"4 1st St, Los Angeles, CA 90001"
67354,Sales_January_2019.csv,141365,Vareebadd Phone,1,400.00,01/10/19 11:19,"20 Dogwood St, New York City, NY 10001"
...,...,...,...,...,...,...,...
55146,Sales_December_2019.csv,319584,Wired Headphones,1,11.99,12/11/19 00:30,"537 Sunset St, Portland, OR 97035"
55158,Sales_December_2019.csv,319596,iPhone,1,700.00,12/01/19 08:44,"436 14th St, New York City, NY 10001"
55159,Sales_December_2019.csv,319596,Lightning Charging Cable,1,14.95,12/01/19 08:44,"436 14th St, New York City, NY 10001"
55195,Sales_December_2019.csv,319631,Lightning Charging Cable,1,14.95,12/17/19 22:35,"363 Highland St, Austin, TX 73301"


### Pregunta 1: ¿Qué mes tuvo las ventas más altas? ¿Cuánto se ganó en dicho periodo?

#### Adición de nuevas columnas

In [ ]:
all_data['Month'] = all_data["Order Date"].str[0:2]
all_data.head()

#### Cambio del tipo de variables

In [ ]:
#all_data["Month"] = all_data["Month"].astype('int32')

In [ ]:
all_data.dtypes

In [ ]:
convert_dic = {"Order ID": "int32","Product": str,"Quantity Ordered": int,
               "Price Each": float,"Order Date": str,"Purchase Address": str,
               "File_origin": str}
all_data = all_data.astype(convert_dic)
print(all_data.dtypes)

#### Adición de la columna de ventas

In [ ]:
all_data["Sales"] = all_data["Quantity Ordered"] * all_data["Price Each"]
all_data.head()

#### Respuesta a la pregunta

In [ ]:
# Diciembre fue el mes con las mejores ventas
results = all_data.groupby("Month").sum()[["Quantity Ordered","Sales"]]
results

In [ ]:
months = range(1,13)
plt.bar(months,results["Sales"])
plt.xticks(months)
plt.ylabel("Ventas en dólares")
plt.xlabel("Número de mes")
plt.show

### ¿Qué ciudad tuvo las ventas más altas? ¿Cuánto se ganó en dicho periodo?

#### Separar columnas

In [ ]:
all_data.head()

In [ ]:
# split column and add new columns to df
all_data2 = all_data.copy()
all_data2[['Address', 'City', 'Code']] = all_data2['Purchase Address'].str.split(',', expand=True)
all_data2.head()

In [ ]:
results2 = all_data2.groupby("City").sum()[["Quantity Ordered","Sales"]]
results2

Es posible aislar solamente la columna de interés

In [ ]:
all_data['City'] = all_data['Purchase Address'].apply(lambda x: x.split(',')[1])
all_data.head()

Considerar que los nombre de las ciudades pueden estar duplicados si no se extrae el estado

In [ ]:
results3 = all_data.groupby("City").sum()[["Quantity Ordered","Sales"]]
results3

El método apply funcionará bien en tanto no se traten excesivas cantidades de datos

In [ ]:
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]

all_data['City'] = all_data['Purchase Address'].apply(lambda x:f"{get_city(x)} ({get_state(x)})")
all_data.head()

In [ ]:
results4 = all_data.groupby("City").sum()[["Quantity Ordered","Sales"]]
results4

In [ ]:
# Este gráfico está errado por el orden de los ejes y las cantidades
cities = all_data["City"].unique()
plt.bar(cities,results4["Sales"])
plt.xticks(cities, rotation = "vertical",size =8)
plt.ylabel("Ventas en dólares")
plt.xlabel("Ciudad")
plt.show()

Como científicos de datos debemos preguntarnos por qué San Francisco tinen las ventas más altas... <br/>
Pueden formularse varias hipótesis que viene de una segmentación de mercado: nivel de ingresos, ubicación, edad, etc.

In [ ]:
keys = [city for city, df in all_data2.groupby(['City'])]
plt.bar(keys,all_data2.groupby(['City']).sum()['Sales'])
plt.ylabel('Ingresos en USD ($)')
plt.xlabel('Ciudad')
plt.xticks(keys, rotation='vertical', size=8)
plt.show()

### ¿Cuál es el mejor momento para desplegar avisos publicitarios de manera que se maximice la probabilidad de que un cliente pueda comprar un producto?

In [ ]:
all_data.head()

In [ ]:
all_data["Order Date"] = pd.to_datetime(all_data["Order Date"])

In [ ]:
all_data.head()

In [ ]:
all_data["Hour"] = all_data["Order Date"].dt.hour
all_data["Minute"] = all_data["Order Date"].dt.minute
all_data.head()

In [ ]:
# Add hour column
all_data['Hour'] = pd.to_datetime(all_data['Order Date']).dt.hour
all_data['Minute'] = pd.to_datetime(all_data['Order Date']).dt.minute
all_data['Count'] = 1
all_data.head()

In [ ]:
keys = [pair for pair, df in all_data.groupby(['Hour'])]

plt.plot(keys, all_data.groupby(['Hour']).count()['Count'])
plt.xticks(keys)
plt.grid()
plt.show()

### ¿Qué productos se venden juntos usualmente?

In [ ]:
all_data.head()

In [ ]:
df = all_data[all_data["Order ID"].duplicated(keep = False)]

df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

df = df[['Order ID','Grouped']].drop_duplicates()

df.head()

In [ ]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))

for key,value in count.most_common(10):
    print(key,value)

### ¿Qué producto vendió más? ¿Por qué fue así?

In [ ]:
all_data.head()

In [ ]:
product_group = all_data.groupby('Product')
quantity_ordered = product_group.sum()['Quantity Ordered']

keys = [pair for pair, df in product_group]
plt.bar(keys, quantity_ordered)
plt.xticks(keys, rotation='vertical', size=8)
plt.show()

In [ ]:
prices = all_data.groupby('Product').mean()['Price Each']
print(prices)

In [ ]:
# Referenced: https://stackoverflow.com/questions/14762181/adding-a-y-axis-label-to-secondary-y-axis-in-matplotlib

prices = all_data.groupby('Product').mean()['Price Each']

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(keys, quantity_ordered, color='g')
ax2.plot(keys, prices, color='b')

ax1.set_xlabel('Product Name')
ax1.set_ylabel('Quantity Ordered', color='g')
ax2.set_ylabel('Price ($)', color='b')
ax1.set_xticklabels(keys, rotation='vertical', size=8)

fig.show()